In [3]:
import pandas as pd
import numpy as np
import os
from ast import literal_eval
from dotenv import load_dotenv
from typing import Dict, Union
from pprint import pprint
from pathlib import Path
from llm_assessor import (
    ranged_error
)

load_dotenv(override=True)

True

In [4]:
# Helper Functions
def extract_total_cost(x: Union[int, Dict]) -> float:

    try: 
        z = literal_eval(x)["total_cost"]
    except ValueError: 
        z = x
    except TypeError:
        z = x
    
    return float(z)

In [7]:
# Set working directory - Not required if using Jupyter outside of VScode
workdir = os.environ["workdir"]

os.chdir(workdir)
os.getcwd()

'/Users/amiralkateb/Documents/GitRepos/audenshaw_exam_validation'

In [8]:
# Load Dataframe
data_dir = Path("./validation_results")

teacher_marked_questions = pd.read_csv(data_dir / "processed_data" / "student_answers_llm_graded.csv")

# Compute performance statistics

In [9]:
teacher_marked_questions_reduced = teacher_marked_questions[~teacher_marked_questions.question_type.isin([])]

In [10]:
# Token Costing
teacher_marked_questions_reduced['llm_grading_total_cost'] = teacher_marked_questions_reduced.apply(lambda x: extract_total_cost(x.llm_graded_answer_token_costing) + extract_total_cost(x.llm_awarded_marks_token_costing), axis=1)

token_cost_by_question_type = teacher_marked_questions_reduced.groupby(by=["subject_id", "question_type"], as_index=False).agg({"llm_grading_total_cost": ["mean", "sum"], "question_id": "count"})
token_cost_by_question_type.columns = ['subject_id', 'question_type', 'mean_llm_grading_total_cost', 'sum_llm_grading_total_cost', 'number_of_questions']

token_cost_by_question_type

,subject_id,question_type,mean_llm_grading_total_cost,sum_llm_grading_total_cost,number_of_questions
0,aqa_history,hs_analyse,0.062370,2.432435,39
1,aqa_history,hs_explain,0.121235,9.456315,78
2,aqa_history,hs_judgement,0.107919,4.208860,39
3,aqa_history,hs_spag,0.012088,0.471445,39
4,edexcel_business_studies,bs_analyse,0.017686,0.919650,52
5,edexcel_business_studies,bs_calculate,0.004580,0.476330,104
6,edexcel_business_studies,bs_discuss,0.014719,0.382705,26
7,edexcel_business_studies,bs_evaluate,0.022879,0.594855,26
8,edexcel_business_studies,bs_explain,0.011141,1.737980,156
9,edexcel_business_studies,bs_identify,0.004802,0.124840,26


In [11]:
# Hitrate
## Mean overall
mean_hitrate_by_subject = teacher_marked_questions_reduced.groupby(by=["subject_id"], as_index = False).agg({"question_id": "count", "llm_mark_hitrate": ["sum", np.nanmean]}).rename(columns={"question_id": "number_of_questions", "llm_mark_hitrate": "proportion_of_questions_covered"})
mean_hitrate_by_subject.columns = ["_".join(col_name).rstrip('_') for col_name in mean_hitrate_by_subject.columns]
mean_hitrate_by_subject

/var/folders/0v/k7t5wdrj1kd3rwp3_7gq20040000gn/T/ipykernel_45174/2604766014.py:3: FutureWarning: The provided callable <function nanmean at 0x107906710> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  mean_hitrate_by_subject = teacher_marked_questions_reduced.groupby(by=["subject_id"], as_index = False).agg({"question_id": "count", "llm_mark_hitrate": ["sum", np.nanmean]}).rename(columns={"question_id": "number_of_questions", "llm_mark_hitrate": "proportion_of_questions_covered"})


,subject_id,number_of_questions_count,proportion_of_questions_covered_sum,proportion_of_questions_covered_nanmean
0,aqa_history,195,50,0.268817
1,edexcel_business_studies,702,411,0.667208


In [12]:
# Same Level hitrate
## Mean overall
mean_same_level_hitrate_by_subject = teacher_marked_questions_reduced.groupby(by=["subject_id"], as_index = False).agg({"question_id": "count", "llm_level_hitrate": ["sum", np.nanmean]}).rename(columns={"question_id": "number_of_questions", "llm_level_hitrate": "proportion_of_questions_covered"})
mean_same_level_hitrate_by_subject.columns = ["_".join(col_name).rstrip('_') for col_name in mean_same_level_hitrate_by_subject.columns]
mean_same_level_hitrate_by_subject

/var/folders/0v/k7t5wdrj1kd3rwp3_7gq20040000gn/T/ipykernel_45174/1586380842.py:3: FutureWarning: The provided callable <function nanmean at 0x107906710> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  mean_same_level_hitrate_by_subject = teacher_marked_questions_reduced.groupby(by=["subject_id"], as_index = False).agg({"question_id": "count", "llm_level_hitrate": ["sum", np.nanmean]}).rename(columns={"question_id": "number_of_questions", "llm_level_hitrate": "proportion_of_questions_covered"})


,subject_id,number_of_questions_count,proportion_of_questions_covered_sum,proportion_of_questions_covered_nanmean
0,aqa_history,195,97.0,0.497436
1,edexcel_business_studies,702,105.0,0.673077


In [13]:
# same level pm1 hitrate
## Mean overall
# mean_same_level_pm1_hitrate_by_subject = teacher_marked_questions_reduced.groupby(by=["subject_id"], as_index = False).agg({"question_id": "count", "same_level_hitrate_pm1": ["sum", np.nanmean]}).rename(columns={"question_id": "number_of_questions", "same_level_hitrate_pm1": "proportion_of_questions_covered"})
# mean_same_level_pm1_hitrate_by_subject.columns = ["_".join(col_name).rstrip('_') for col_name in mean_same_level_pm1_hitrate_by_subject.columns]
# mean_same_level_pm1_hitrate_by_subject

In [14]:
# Hitrate
## Mean by question type
mean_hitrate_by_question_type = teacher_marked_questions_reduced.groupby(by=["subject_id", "question_type"], as_index = False).agg({"question_id": "count", "llm_mark_hitrate": ["sum", np.nanmean]}).rename(columns={"question_id": "number_of_questions", "llm_mark_hitrate": "proportion_of_questions_covered"})
mean_hitrate_by_question_type.columns = ["_".join(col_name).rstrip('_') for col_name in mean_hitrate_by_question_type.columns]

mean_hitrate_by_question_type

/var/folders/0v/k7t5wdrj1kd3rwp3_7gq20040000gn/T/ipykernel_45174/884379214.py:3: FutureWarning: The provided callable <function nanmean at 0x107906710> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  mean_hitrate_by_question_type = teacher_marked_questions_reduced.groupby(by=["subject_id", "question_type"], as_index = False).agg({"question_id": "count", "llm_mark_hitrate": ["sum", np.nanmean]}).rename(columns={"question_id": "number_of_questions", "llm_mark_hitrate": "proportion_of_questions_covered"})


,subject_id,question_type,number_of_questions_count,proportion_of_questions_covered_sum,proportion_of_questions_covered_nanmean
0,aqa_history,hs_analyse,39,12,0.333333
1,aqa_history,hs_explain,78,13,0.171053
2,aqa_history,hs_judgement,39,4,0.108108
3,aqa_history,hs_spag,39,21,0.567568
4,edexcel_business_studies,bs_analyse,52,17,0.425
5,edexcel_business_studies,bs_calculate,104,71,0.771739
6,edexcel_business_studies,bs_discuss,26,6,0.26087
7,edexcel_business_studies,bs_evaluate,26,7,0.411765
8,edexcel_business_studies,bs_explain,156,81,0.536424
9,edexcel_business_studies,bs_identify,26,20,0.952381


In [15]:
# Hitrate by Question ID
teacher_marked_questions_reduced.head()

question_hit_rate = teacher_marked_questions_reduced.groupby(by=['subject_id','question_id', "question_type", "total_marks"], as_index=False).agg({"llm_mark_hitrate": ["count", "sum", np.nanmean]})
question_hit_rate.columns = ["subject_id","question_id", "question_type", "total_marks", "number_of_questions", "hitrate_frequency", "mean_hitrate"]

question_hit_rate = question_hit_rate.sort_values(by=["subject_id","question_id","question_type", "mean_hitrate"], ascending=[True, True, True, True])
question_hit_rate

/var/folders/0v/k7t5wdrj1kd3rwp3_7gq20040000gn/T/ipykernel_45174/642752629.py:4: FutureWarning: The provided callable <function nanmean at 0x107906710> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  question_hit_rate = teacher_marked_questions_reduced.groupby(by=['subject_id','question_id', "question_type", "total_marks"], as_index=False).agg({"llm_mark_hitrate": ["count", "sum", np.nanmean]})


,subject_id,question_id,question_type,total_marks,number_of_questions,hitrate_frequency,mean_hitrate
0,aqa_history,1.0,hs_explain,4,37,6,0.162162
1,aqa_history,2.0,hs_explain,12,39,7,0.179487
2,aqa_history,3.0,hs_analyse,8,36,12,0.333333
3,aqa_history,4.1,hs_judgement,16,37,4,0.108108
4,aqa_history,4.2,hs_spag,4,37,21,0.567568
5,edexcel_business_studies,1.1,bs_mcq,1,26,25,0.961538
6,edexcel_business_studies,1.2,bs_mcq,1,26,26,1.0
7,edexcel_business_studies,1.3,bs_explain,3,25,18,0.72
8,edexcel_business_studies,1.4,bs_explain,3,26,11,0.423077
9,edexcel_business_studies,2.1,bs_mcq,2,26,26,1.0


In [16]:
# Mean Absolute Error
teacher_marked_questions_reduced["marks_mean_absolute_error"] = teacher_marked_questions_reduced.apply(lambda row: ranged_error(x=row['llm_awarded_marks'], range_of_values=[row['awarded_marks']]), axis=1)

# Overall
marks_mea = np.mean(teacher_marked_questions_reduced.marks_mean_absolute_error)

## Mean by question type
marks_mea_by_question_type = teacher_marked_questions_reduced.groupby(by=["subject_id", "question_type"], as_index = False).agg({"question_id": "count", "marks_mean_absolute_error": "mean"}).rename(columns={"question_id": "number_of_questions"})
marks_mea_by_question_type = marks_mea_by_question_type.sort_values(by=["subject_id", "question_type", "number_of_questions"], ascending=[True, True, True])

print(f"Overall Mean:  {marks_mea:.2f}")
marks_mea_by_question_type

Overall Mean:  0.57


,subject_id,question_type,number_of_questions,marks_mean_absolute_error
0,aqa_history,hs_analyse,39,0.948718
1,aqa_history,hs_explain,78,1.256410
2,aqa_history,hs_judgement,39,2.000000
3,aqa_history,hs_spag,39,0.641026
4,edexcel_business_studies,bs_analyse,52,0.596154
5,edexcel_business_studies,bs_calculate,104,0.192308
6,edexcel_business_studies,bs_discuss,26,0.846154
7,edexcel_business_studies,bs_evaluate,26,0.653846
8,edexcel_business_studies,bs_explain,156,0.602564
9,edexcel_business_studies,bs_identify,26,0.038462


In [17]:
# Mean scaled error metric

# Overall
scaled_metric_mean = np.nanmean(teacher_marked_questions_reduced.scaled_error_metric)

## Mean by question type
scaled_metric_mean_by_question_type = teacher_marked_questions_reduced.groupby(by=["subject_id", "question_type"], as_index = False).agg({"question_id": "count", "scaled_error_metric": "mean"}).rename(columns={"question_id": "number_of_questions"})
scaled_metric_mean_by_question_type = scaled_metric_mean_by_question_type.sort_values(by=["subject_id", "question_type", "number_of_questions"], ascending=[True, True, True])

print(f"Overall Mean:  {scaled_metric_mean:.2f}")
scaled_metric_mean_by_question_type

Overall Mean:  0.10


,subject_id,question_type,number_of_questions,scaled_error_metric
0,aqa_history,hs_analyse,39,0.074603
1,aqa_history,hs_explain,78,0.152628
2,aqa_history,hs_judgement,39,0.120767
3,aqa_history,hs_spag,39,0.114138
4,edexcel_business_studies,bs_analyse,52,0.071528
5,edexcel_business_studies,bs_calculate,104,NaN
6,edexcel_business_studies,bs_discuss,26,0.086325
7,edexcel_business_studies,bs_evaluate,26,0.026191
8,edexcel_business_studies,bs_explain,156,NaN
9,edexcel_business_studies,bs_identify,26,NaN


In [18]:
# Mean Absolute Error by question deviation
question_type_mark_deviations = teacher_marked_questions_reduced.groupby(by=["question_type", "marks_mean_absolute_error"], as_index = False).agg({"question_id": "count"})
question_type_totals = teacher_marked_questions_reduced.groupby(by=["subject_id", "question_type"], as_index = False).agg({"question_id": "count"})

question_type_mark_deviation_joined = pd.merge(question_type_mark_deviations, question_type_totals, on = "question_type", how="left")
question_type_mark_deviation_joined['questions_proportion'] = question_type_mark_deviation_joined.apply(lambda x: x['question_id_x']/ x['question_id_y'], axis = 1)

question_type_mark_deviation_joined = question_type_mark_deviation_joined.rename(columns={"question_id_x": "number_of_questions", "question_id_y": "number_of_questions_by_question_type"})
ordered_cols = ["subject_id", "question_type", "marks_mean_absolute_error", "number_of_questions", "number_of_questions_by_question_type", "questions_proportion"]
question_type_mark_deviation_joined = question_type_mark_deviation_joined[ordered_cols]
question_type_mark_deviation_joined = question_type_mark_deviation_joined.sort_values(by=["subject_id", "question_type", "marks_mean_absolute_error"], ascending=[True, True, True]).reset_index(drop=True)

question_type_mark_deviation_joined

,subject_id,question_type,marks_mean_absolute_error,number_of_questions,number_of_questions_by_question_type,questions_proportion
0,aqa_history,hs_analyse,0.0,15,39,0.384615
1,aqa_history,hs_analyse,1.0,13,39,0.333333
2,aqa_history,hs_analyse,2.0,10,39,0.256410
3,aqa_history,hs_analyse,4.0,1,39,0.025641
4,aqa_history,hs_explain,0.0,15,78,0.192308
5,aqa_history,hs_explain,1.0,33,78,0.423077
6,aqa_history,hs_explain,2.0,26,78,0.333333
7,aqa_history,hs_explain,3.0,3,78,0.038462
8,aqa_history,hs_explain,4.0,1,78,0.012821
9,aqa_history,hs_judgement,0.0,6,39,0.153846


In [23]:
overall_stats = teacher_marked_questions_reduced.groupby(by=["subject_id"], as_index=False).agg({
    "llm_mark_hitrate": 'mean', 
    'llm_level_hitrate': 'mean', 
    #'same_level_hitrate_pm1': 'mean', 
    'scaled_error_metric': 'mean', 
    'question_id': 'count', 
    'elapsed_time_in_seconds': 'sum', 
    'llm_grading_total_cost': 'sum'})

overall_stats['mean_hitrate'] = overall_stats['llm_mark_hitrate'].apply(lambda x: np.round(x, 2))
overall_stats['mean_llm_level_hitrate'] = overall_stats['llm_level_hitrate'].apply(lambda x: np.round(x, 2))
#overall_stats['mean_same_level_hitrate_pm1'] = overall_stats['same_level_hitrate_pm1'].apply(lambda x: np.round(x, 2))
overall_stats['mean_scaled_error_metric'] = overall_stats['scaled_error_metric'].apply(lambda x: np.round(x, 2))


overall_stats['number_of_questions'] = overall_stats.question_id
overall_stats['elapsed_time_in_minutes'] = overall_stats['elapsed_time_in_seconds'].apply(lambda x: x/ 60)
overall_stats['llm_grading_total_cost'] = overall_stats['llm_grading_total_cost'].apply(lambda x: np.round(x, 2))

overall_stats = overall_stats[[
    'subject_id', 
    'mean_hitrate', 
    'mean_llm_level_hitrate', 
    #'mean_same_level_hitrate_pm1', 
    'mean_scaled_error_metric', 
    'number_of_questions', 
    'elapsed_time_in_minutes', 
    'llm_grading_total_cost']]

overall_stats

,subject_id,mean_hitrate,mean_llm_level_hitrate,mean_scaled_error_metric,number_of_questions,elapsed_time_in_minutes,llm_grading_total_cost
0,aqa_history,0.27,0.50,0.12,195,12.040275,16.57
1,edexcel_business_studies,0.67,0.67,0.07,702,31.063821,6.90


In [24]:
# Save validation statistics tables
savedir = Path(workdir) / "validation_results" / "validation_statistics"

Path(savedir).mkdir(parents=True, exist_ok=True)

In [25]:
teacher_marked_questions.to_csv(savedir / "examiner_llm_comparison.csv", index=False)

In [26]:
overall_stats.to_csv(savedir / "mean_hitrate.csv", index=False)
mean_hitrate_by_question_type.to_csv(savedir / "mean_hitrate_by_question_type.csv", index=False)
marks_mea_by_question_type.to_csv(savedir / "mea_marks_by_question_type.csv", index=False)
question_type_mark_deviation_joined.to_csv(savedir / "question_type_marks_deviation.csv", index=False)
question_hit_rate.to_csv(savedir / "question_hit_rate.csv", index=False)
token_cost_by_question_type.to_csv(savedir / "token_cost_by_question_type.csv", index=False)
scaled_metric_mean_by_question_type.to_csv(savedir / "scaled_metric_mean_by_question_type.csv", index = False)
mean_same_level_hitrate_by_subject.to_csv(savedir / "mean_same_level_hitrate_by_subject.csv", index = False)
#mean_same_level_pm1_hitrate_by_subject.to_csv(savedir / "mean_same_level_pm1_hitrate_by_subject.csv", index = False)